In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('binary_class').getOrCreate()
df=spark.read.csv('BankChurners.csv',inferSchema=True,header=True)

In [3]:
print((df.count(),len(df.columns)))
df.printSchema()

(10127, 21)
root
 |-- CLIENTNUM: integer (nullable = true)
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: integer (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: integer (nullable = true)
 |-- Total_Relationship_Count: integer (nullable = true)
 |-- Months_Inactive_12_mon: integer (nullable = true)
 |-- Contacts_Count_12_mon: integer (nullable = true)
 |-- Credit_Limit: double (nullable = true)
 |-- Total_Revolving_Bal: integer (nullable = true)
 |-- Avg_Open_To_Buy: double (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: double (nullable = true)
 |-- Total_Trans_Amt: integer (nullable = true)
 |-- Total_Trans_Ct: integer (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: double (nullable = true)
 |-- Avg_Utilization_Rati

In [4]:
df.select("Gender","Card_Category").describe().show(truncate=False)

+-------+------+-------------+
|summary|Gender|Card_Category|
+-------+------+-------------+
|count  |10127 |10127        |
|mean   |null  |null         |
|stddev |null  |null         |
|min    |F     |Blue         |
|max    |M     |Silver       |
+-------+------+-------------+



In [5]:
#Features process
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
loan_purpose_indexer = StringIndexer(inputCol="loan_purpose", outputCol="loan_index").fit(df)
df = loan_purpose_indexer.transform(df)
loan_encoder = OneHotEncoder(inputCol="loan_index",outputCol="loan_purpose_vec").fit(df)
df = loan_encoder.transform(df)


Py4JJavaError: An error occurred while calling o42.fit.
: org.apache.spark.SparkException: Input column loan_purpose does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:245)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:245)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:242)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.spa

In [ ]:
#Logestic Regression
from pyspark.ml.classification import LogisticRegression
training_df,test_df=model_df.randomSplit([0.75,0.25])
log_reg=LogisticRegression(regParam=0.01).fit(training_df)
lr_summary=log_reg.summary
lr_summary.accuracy
lr_summary.areaUnderROC
predictions = log_reg.transform(test_df)
predictions.show(10)
predictions = log_reg.evaluate(test_df)

#Decision Tree

from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(maxDepth=5)
dtModel = dt.fit(training_df)
result = dtModel.transform(test_df)

#Random Forest
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(numTrees=10, maxDepth=5)
rfModel = rf.fit(training_df)
result = rfModel.transform(test_df)

#https://mp.weixin.qq.com/s/fPT4idMJph7Gru9RqTFbYQ